In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("emergency.db")
print("Database created")

Database created


In [3]:
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS emergency_events (
    event_id TEXT PRIMARY KEY,
    user_id TEXT NOT NULL,
    emergency_id TEXT NOT NULL,
    event_type TEXT NOT NULL,
    timestamp INTEGER NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS pogo_scores (
    emergency_id TEXT PRIMARY KEY,
    user_id TEXT NOT NULL,
    pogo_score REAL NOT NULL,
    calculated_at INTEGER NOT NULL
)
""")

conn.commit()
print("Tables created")


Tables created


In [13]:
# Event constants
CREATE = "create_emergency"
CANCEL = "cancel_emergency"
REOPEN = "reopen_emergency"
RESOLVE = "resolve_emergency"
UPDATE = "update_emergency"
CHANGE_CATEGORY = "change_category"
AUTO_RESOLVE = "auto_resolve"


In [14]:
import uuid, time

def log_event(user_id, emergency_id, event_type):
    cursor.execute(
        "INSERT INTO emergency_events VALUES (?, ?, ?, ?, ?)",
        (
            str(uuid.uuid4()),
            user_id,
            emergency_id,
            event_type,
            int(time.time())
        )
    )
    conn.commit()


In [16]:
user_id = "user_1"
emergency_id = "emg_1"

log_event(user_id, emergency_id, CREATE)
log_event(user_id, emergency_id, CANCEL)


In [17]:
cursor.execute("SELECT * FROM emergency_events")
cursor.fetchall()


[('dad96048-8cd1-4e0d-a61c-1f72275921dc',
  'user_1',
  'emg_1',
  'create_emergency',
  1768645569),
 ('2080c64d-871b-4a27-81e0-3261789d9070',
  'user_1',
  'emg_1',
  'create_emergency',
  1768645631),
 ('37c33b62-790e-4adc-b07c-a3cf07735fd2',
  'user_1',
  'emg_1',
  'cancel_emergency',
  1768645631),
 ('ea260013-d739-465c-8cab-9636408b98f9',
  'user_1',
  'emg_2',
  'create_emergency',
  1768645631),
 ('8198b159-b1be-45a6-a22b-0fbd8c8ab975',
  'user_1',
  'emg_1',
  'create_emergency',
  1768646293),
 ('04849725-4a3b-4038-b695-f15815c972ec',
  'user_1',
  'emg_1',
  'cancel_emergency',
  1768646293)]

In [7]:
def get_recent_events(user_id, seconds=300):
    cutoff = int(time.time()) - seconds
    cursor.execute(
        """
        SELECT event_type FROM emergency_events
        WHERE user_id = ?
        AND timestamp >= ?
        """,
        (user_id, cutoff)
    )
    return [row[0] for row in cursor.fetchall()]


In [18]:
def compute_pogo_score(events):
    submits = events.count("create_emergency")
    cancels = events.count("cancel_emergency")
    reopens = events.count("reopen_emergency")

    if submits == 0:
        return 0.0

    cancel_rate = cancels / submits

    pogo_score = (
        0.5 * min(submits / 3, 1) +
        0.3 * cancel_rate +
        0.2 * min(reopens / 2, 1)
    )

    return round(min(pogo_score, 1.0), 2)


In [19]:
def save_pogo_score(user_id, emergency_id, score):
    cursor.execute(
        """
        INSERT OR REPLACE INTO pogo_scores
        VALUES (?, ?, ?, ?)
        """,
        (
            emergency_id,
            user_id,
            score,
            int(time.time())
        )
    )
    conn.commit()


In [20]:
log_event("user_1", "emg_1", "create_emergency")
log_event("user_1", "emg_1", "cancel_emergency")
log_event("user_1", "emg_2", "create_emergency")

events = get_recent_events("user_1")
score = compute_pogo_score(events)
save_pogo_score("user_1", "emg_2", score)

print("Pogo score:", score)


Pogo score: 0.7


In [21]:
cursor.execute("SELECT * FROM pogo_scores")
cursor.fetchall()


[('emg_2', 'user_1', 0.7, 1768646316)]